# Sistema de Compressão

$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^{-3} m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 MPa \\
P{out} & = & 5.0 MPa \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

#### Importações

In [2]:
import numpy as np
from scipy.optimize import fsolve
import casadi as ca
import plotly.graph_objects as go
import optuna
from plotly.subplots import make_subplots
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


/home/matheus/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


#### Constantes e Variáveis Auxiliares

In [3]:
np.random.seed(42)
print(np.random.seed)

# Constantes
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 3 # Passos no passado para prever o próximo
epochs = 5000
nData = 1000 
nAlphas = 5 # Número de vezes que o Alfa irá mudar, considere o treino e os testes.
alphas = np.random.uniform(0.35,0.65, nAlphas+1) # Abertura da válvula
perturb = 1e-3
tempo = 100
dt = 0.1 # Tempo amostral

# Variáveis auxiliares
interval = [np.linspace(i * tempo, (i + 1) * tempo, nData) for i in range(nAlphas)]
interval_test = [np.linspace(i * 60, (i + 1) * 60, 600) for i in range(nAlphas)]
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []

<built-in function seed>


### Solução Numérica

##### Cálculo da Solução

In [4]:


# Condições Iniciais
def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)*1e3
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y*1000 - P_out*1000))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0] 
init_p = result[1]

# Variáveis CasADi
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)

# Solução Numérica
tm1 = time.time()
for i in range(nAlphas):
    alpha_values.append(alphas[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[0][0], dt)

    for j in range(nData):
        sol = F(x0 = [init_m, init_p], p = alpha_values[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrate.append(aux1)
        PlenumPressure.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_train.append([aux1[0], aux2[0], alpha_values[i][j]])
        RNN_trainFut.append([aux1[0], aux2[0], alpha_values[i][j]])
    
tm2 = time.time()

massFlowrate = np.reshape(massFlowrate, [nAlphas,nData])
PlenumPressure = np.reshape(PlenumPressure, [nAlphas,nData])

print(tm2-tm1)
print(RNN_train)


2.947629690170288
[[7.351447130061303, 6.738811494495828, 0.4639412484697161], [7.352084177557419, 6.745214893828777, 0.46312947038336166], [7.351511918157013, 6.754301378852157, 0.46189256126827377], [7.3519328819603516, 6.746839030471486, 0.4629045956977947], [7.351374817106706, 6.75418994105393, 0.4618986179613963], [7.350848539425641, 6.753956347001726, 0.4618963059006385], [7.351024419553049, 6.748677706600474, 0.4626039979257748], [7.3491293283453265, 6.764176323286021, 0.46044875540955094], [7.347533337543864, 6.761968275363956, 0.4606371178216957], [7.347145397866058, 6.752922413854528, 0.46179974812496777], [7.34635053935904, 6.7559678209887135, 0.46134920453387435], [7.346863088282142, 6.746152707981595, 0.462676282986804], [7.346185695100297, 6.75509149905481, 0.46145401157868754], [7.345066736977126, 6.758397958551045, 0.4609497319528735], [7.3467428111535265, 6.73743729108393, 0.4638276844231303], [7.346720525775621, 6.750168068723585, 0.4621362593537222], [7.3469770452756

##### Gráfico do Modelo

In [5]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Vazão vs Tempo", "Pressão vs Tempo", "Alpha vs Tempo"))

for i in range(0, nAlphas):
    # Vazão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Vazão', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    # Pressão
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Pressão', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    # Alphas
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alphas', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    xaxis_title='Tempo',
    grid=dict(rows=1, columns=3),
    template='plotly',
    showlegend=False
)

# Mostra a figura
fig.show()


### Rede Neural

##### Dados de Treino

In [7]:
newData = int((nData*nAlphas)/(timestep + 1))
RNN_train = np.array(RNN_train)


X_train = []
y_train = []

for i in range(len(RNN_train) - timestep):
    X_train.append(RNN_train[i:i + timestep])  
    if i + timestep < len(RNN_train):           
        y_train.append(RNN_train[i + timestep, :2])  

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

y_train = y_train.unsqueeze(1)

##### Rede

In [63]:
class MyModel(nn.Module):
    def __init__(self, units):
        super(MyModel, self).__init__()
        
        # Camadas RNN
        self.input_layer = nn.Linear(3, 3)
        self.rnn_layers = nn.ModuleList([
            nn.RNN(input_size=3, hidden_size=units, batch_first=True, bidirectional=True)
        ])
        self.rnn_layers.append(nn.RNN(input_size=units*2, hidden_size=units, batch_first=True))
        self.dense = nn.Linear(units, 2)
    
    def forward(self, inputs):
        # Passagem pelas camadas RNN
        rnn_output = inputs
        for rnn_layer in self.rnn_layers:
            rnn_output, _ = rnn_layer(rnn_output)
        
        # Saída densa
        dense_output = self.dense(rnn_output[:, -1, :])  # Pegando apenas a última saída da sequência
        return dense_output

def loss_custom(y_true, y_pred, inputs):
    # Implementação da função de perda
    m_t = (11* y_pred[:, 0] - 18 *inputs[:, -2, 0] + 9 * inputs[:, -3, 0] - 2 * inputs[:, 0, 0])/6*dt
    p_t = (11* y_pred[:, 1] - 18 *inputs[:, -2, 1] + 9 * inputs[:, -2, 1] - 2 * inputs[:, -2, 1])/6*dt
    fLoss_mass = torch.mean(torch.square(m_t - (A1/Lc)*((1.5 * P1) - y_pred[:, 1]) * 1e3))
    fLoss_pres = torch.mean(torch.square(p_t - (C**2)/2 * (y_pred[:, 0] - inputs[:, -1, -1]* kv * torch.sqrt((torch.abs(y_pred[:, 1] * 1000 - P_out * 1000))))))
    phys_loss = fLoss_mass + fLoss_pres
    data_loss = torch.mean((y_true[:, 0, 0] - y_pred[:, 0]) ** 2) + 1e3 * torch.mean((y_true[:, 0, 1] - y_pred[:, 1]) ** 2)
    return data_loss + 1e-10* phys_loss

def train_model(model, train_loader, lr, epochs):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        for inputs, y_true in train_loader:
            optimizer.zero_grad()
            
            y_pred = model(inputs)
            loss = loss_custom(y_true, y_pred, inputs)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader)}")




##### Treinamento

In [ ]:
def objective(trial):
    lr = trial.suggest_float('lr', 1e-6, 1e-3, log=True) 
    units = trial.suggest_int('units', 32, 64)
    batch_size = trial.suggest_int('batch_size', 16, 64) 
    num_layers = trial.suggest_int('num_layers', 0, 5)  

    model = create_model(lr, num_layers, units)

    model.fit(X_train, y_train, epochs=250, batch_size=batch_size, verbose=0)

    # Avaliação
    loss = model.evaluate(X_train, y_train, verbose=0)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)  # Ajuste o número de tentativas conforme necessário

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros: ", study.best_params)

In [69]:
model = MyModel(units = 30)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size= 128, shuffle=True)


train_model(model, train_loader, 0.0001, 2000)

Epoch [1/2000], Loss: 44598.7171875
Epoch [2/2000], Loss: 35729.71762695313
Epoch [3/2000], Loss: 28216.445654296876
Epoch [4/2000], Loss: 22151.82880859375
Epoch [5/2000], Loss: 17611.97490234375
Epoch [6/2000], Loss: 14344.042236328125
Epoch [7/2000], Loss: 11959.642211914062
Epoch [8/2000], Loss: 10155.732250976562
Epoch [9/2000], Loss: 8745.342919921875
Epoch [10/2000], Loss: 7644.0428466796875
Epoch [11/2000], Loss: 6748.91943359375
Epoch [12/2000], Loss: 5988.781652832031
Epoch [13/2000], Loss: 5339.457861328125
Epoch [14/2000], Loss: 4771.966638183594
Epoch [15/2000], Loss: 4270.576922607422
Epoch [16/2000], Loss: 3823.6098571777343
Epoch [17/2000], Loss: 3426.0510131835936
Epoch [18/2000], Loss: 3071.9389587402343
Epoch [19/2000], Loss: 2752.565380859375
Epoch [20/2000], Loss: 2456.2535858154297
Epoch [21/2000], Loss: 2205.1328125
Epoch [22/2000], Loss: 1971.4181854248047
Epoch [23/2000], Loss: 1758.4203674316407
Epoch [24/2000], Loss: 1566.9499938964843
Epoch [25/2000], Loss: 

##### Dados de teste

In [70]:
massFlowrateTeste = []
PlenumPressureTeste = []
RNN_test = []
x_test = []
alpha_valuesTeste = []
alphasTeste = np.random.uniform(0.35,0.65, nAlphas) # Abertura da válvula
aux1 = []
aux2 = []

tm1 = time.time()
for i in range(0,nAlphas):
    alpha_valuesTeste.append(alphasTeste[i] + np.random.normal(0,perturb,nData))

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1])*1e3, (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1]*1000 - P_out*1000)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }
    
    for j in range(600):
        F = ca.integrator('F','idas', ode, interval[0][0], dt)
        sol = F(x0 = [init_m, init_p], p = alpha_valuesTeste[i][j])
        xf_values = np.array(sol["xf"])
        aux1, aux2 = xf_values
        massFlowrateTeste.append(aux1)
        PlenumPressureTeste.append(aux2)
        init_m = aux1[-1]
        init_p = aux2[-1]
        RNN_test.append([aux1[0], aux2[0], alpha_valuesTeste[i][j]])

tm2 = time.time()
print(tm2-tm1)

RNN_test = np.array(RNN_test)

for i in range(len(RNN_test) - 3):
    x_test.append(RNN_test[i:i + 3]) 

x_test = torch.tensor(x_test, dtype=torch.float32)
print(x_test[:, 0, -1])
massFlowrateTeste = np.array(massFlowrateTeste)
PlenumPressureTeste = np.array(PlenumPressureTeste)
massFlowrateTeste = np.reshape(massFlowrateTeste, [nAlphas, 600])
PlenumPressureTeste = np.reshape(PlenumPressureTeste, [nAlphas, 600])

3.506241798400879
tensor([0.5699, 0.5682, 0.5708,  ..., 0.3694, 0.3725, 0.3714])


##### Gráfico Rede Neural

In [71]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))


tm1 = time.time()
# for i in range(newData):
    # inputs = np.array([temp1,temp2])
    # prediction = model.predict(inputs)
    # temp1 = x_test[:,1,:]
    # temp2 = prediction[0,:,:]
    # mass.append(prediction[:,0,:])
    # pressure.append(prediction[:,0,:])

# Colocando o modelo em modo de avaliação
model.eval()

# Supondo que x_test já esteja definido e seja um tensor PyTorch
with torch.no_grad():  # Desabilitar o cálculo de gradientes
    prediction = model(x_test)

mass = prediction[:, 0]
pressure = prediction[:, 1]

mass = mass.detach().numpy()
pressure = pressure.detach().numpy()

tm2 = time.time()
print(tm2-tm1)

interval2 = [np.linspace(0, tempo*nAlphas, newData) for i in range(nAlphas)]

interval3 = np.linspace(0, 300, 2997)

fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(mass),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig.add_trace(go.Scatter(x=interval3,y=np.squeeze(pressure),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)


for i in range(nAlphas):
    '''
    for j in range(0, len(interval2[i]), nData):
        end = j + nData
        if end > len(interval2[i]):
            end = len(interval2[i])
        
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(mass[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
        fig.add_trace(go.Scatter(x=interval2[i][j:end],y=np.squeeze(pressure[j:end]),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)
    '''
    # Modelo
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig.show()


0.0058825016021728516


In [72]:
model.eval()

massFlowrate100 = [x_test[0, 0, 0].item(), x_test[0, 1, 0].item(), x_test[0, 2, 0].item()]
PlenumPressure100 = [x_test[0, 0, 1].item(), x_test[0, 1, 1].item(), x_test[0, 2, 1].item()]

# Loop para fazer previsões
for i in range(2997):
    # Criação do tensor de entrada
    inputs = torch.tensor([[
        [massFlowrate100[-3], PlenumPressure100[-3], x_test[i, 0, 2].item()],
        [massFlowrate100[-2], PlenumPressure100[-2], x_test[i, 1, 2].item()],
        [massFlowrate100[-1], PlenumPressure100[-1], x_test[i, 2, 2].item()]
    ]], dtype=torch.float32)
    # Desabilitar o cálculo de gradientes
    with torch.no_grad():
        prediction100 = model(inputs)  # Chamar o modelo diretamente para previsões

    # Armazenar as previsões
    massFlowrate100.append(prediction100[0, 0].item())  # Converte tensor para valor escalar  # Imprime a previsão de fluxo de massa
    PlenumPressure100.append(prediction100[0, 1].item())


In [73]:
fig3 = make_subplots(rows=1, cols=3, subplot_titles=("Loss","Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(massFlowrate100),mode='lines',
                                line=dict(dash='solid')),row=1, col=2)
fig3.add_trace(go.Scatter(x=interval3,y=np.squeeze(PlenumPressure100),mode='lines',
                                line=dict(dash='solid')),row=1, col=3)



for i in range(nAlphas):
    # Modelo
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(massFlowrateTeste[i]), mode='lines',name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)
    fig3.add_trace(go.Scatter(x=np.squeeze(interval_test[i]), y=np.squeeze(PlenumPressureTeste[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

fig3.update_layout(
    title='Resultados Rede Neural',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
    showlegend=False
)
fig3.show()